# Skript for Live Vehicle Data Cleaning
## Creating the Client Connection to the Cloud Obejct Storage and the "smart-city-live-vehicle-positions" bucket

The following code cell can be automatically inserted trough the Notebook UI. To do so, click on the data button (top right corner) there you find the *files* and *connections* tab. Go to the *connection* as we want to create a client to our Cloud Object Storage. 

There you will find the Connection which we created before. Click "insert to code" and choose the "StreamingBody object" option. After that there will open a pop up which showes you the folder structure of your underlying cloud bucket. Choose the right folders and subfolders until you end up in the last subfolder, that contains all the .json files we need. Choose one file and click *Select*. Next you will see a code cell, inserted automatically, that looks like this one except it contains the correct api-keys etc.

> It doesn't matter which .json you will choose, because we will later on only use the created client object to access more then only one .json file.

In [1]:
# @hidden_cell


# This connection object is used to access your data and contains your credentials or project token.
# You might want to remove those credentials before you share your notebook.


import types
import pandas as pd
import ibm_boto3
from botocore.client import Config

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.

Cloud_Object_Storage_Connection_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id='api-key',
    ibm_service_instance_id='service-instance-id',
    ibm_auth_endpoint='https://iam.cloud.ibm.com/identity/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-de.cloud-object-storage.appdomain.cloud'
)

body = Cloud_Object_Storage_Connection_client.get_object(Bucket='smart-city-live-vehicle-positions', Key='topics/open_HFP_API/partition=0/open_HFP_API+0+0054722848.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face an error during data loading.
# Refer to the documentation of 'pandas.read_json()' and 'pandas.io.json.json_normalize' for more possibilities to adjust the data loading.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html


After we sucsessfully created the client (your's might be named differently than `Cloud_Object_Storage_Connection_client`, either change it in the cell above or keep in mind to change the name whenever the client is used) we now need a function that can read/get/access more than one .json file. 

As we know, the data is saved in a S3 object style. The following cell shows the function we use to access files over a given timespan in which they were written to the storage. (Don't stress about the function and how it works in detail :joy: ) Just copy and paste it.


In [2]:
import argparse
import boto3
import dateutil.parser
import logging
import pytz
from collections import namedtuple

import pandas as pd
from datetime import datetime, timezone, timedelta



logger = logging.getLogger(__name__)


Rule = namedtuple('Rule', ['has_min', 'has_max'])
last_modified_rules = {
    Rule(has_min=True, has_max=True):
        lambda min_date, date, max_date: min_date <= date <= max_date,
    Rule(has_min=True, has_max=False):
        lambda min_date, date, max_date: min_date <= date,
    Rule(has_min=False, has_max=True):
        lambda min_date, date, max_date: date <= max_date,
    Rule(has_min=False, has_max=False):
        lambda min_date, date, max_date: True,
}

def get_s3_objects(s3, bucket, prefixes=None, suffixes=None, last_modified_min=None, last_modified_max=None):
    """
    Generate the objects in an S3 bucket. Adapted from:
    https://alexwlchan.net/2017/07/listing-s3-keys/

    :param bucket: Name of the S3 bucket.
    :ptype bucket: str
    :param prefixes: Only fetch keys that start with these prefixes (optional).
    :ptype prefixes: tuple
    :param suffixes: Only fetch keys that end with thes suffixes (optional).
    :ptype suffixes: tuple
    :param last_modified_min: Only yield objects with LastModified dates greater than this value (optional).
    :ptype last_modified_min: datetime.date
    :param last_modified_max: Only yield objects with LastModified dates greater than this value (optional).
    :ptype last_modified_max: datetime.date

    :returns: generator of dictionary objects
    :rtype: dict https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.list_objects
    """
    if last_modified_min and last_modified_max and last_modified_max < last_modified_min:
        raise ValueError(
            "When using both, last_modified_max: {} must be greater than last_modified_min: {}".format(
                last_modified_max, last_modified_min
            )
        )
    # Use the last_modified_rules dict to lookup which conditional logic to apply
    # based on which arguments were supplied
    last_modified_rule = last_modified_rules[bool(last_modified_min), bool(last_modified_max)]

    if not prefixes:
        prefixes = ('',)
    else:
        prefixes = tuple(set(prefixes))
    if not suffixes:
        suffixes = ('',)
    else:
        suffixes = tuple(set(suffixes))

    kwargs = {'Bucket': bucket}

    for prefix in prefixes:
        kwargs['Prefix'] = prefix
        while True:
            # The S3 API response is a large blob of metadata.
            # 'Contents' contains information about the listed objects.
            resp = s3.list_objects_v2(**kwargs)
            for content in resp.get('Contents', []):
                last_modified_date = content['LastModified']
                if (
                    content['Key'].endswith(suffixes) and
                    last_modified_rule(last_modified_min, last_modified_date, last_modified_max)
                ):
                    yield content

            # The S3 API is paginated, returning up to 1000 keys at a time.
            # Pass the continuation token into the next response, until we
            # reach the final page (when this field is missing).
            try:
                kwargs['ContinuationToken'] = resp['NextContinuationToken']
            except KeyError:
                break

## Defining the Timespan, which jsons/S3 objects will be collected

As introduced right before this, we are able to access data from LVD trough our client and define a timespane in which we want to get the data.

We decided to create three variables:
`dateloading`,
`starttime`,
`endtime`, to create the timespan we were talking about. 

> Even for a few hours the data that has been collected can sum up to 1.000.000+ rows. So to get a feeling of the data cleaning process it is more than enough to create a small timespan (one hour). Also do remember that it is possible, if you change date and time, that there is no data available.

In [4]:
dateloading = "2022-02-26"
starttime = datetime.fromisoformat(dateloading + ' 12:00:00.000+00:00')
endtime = datetime.fromisoformat(dateloading + ' 13:00.000+00:00')

Because the data amount is so large, we needed to access the data day by day and within that day in a few hour steps. So we needed the following cell to create a recognizable filename at the end.

In [5]:
filename_starttime = starttime.hour
filename_endtime = endtime.hour

The 

In [6]:
objs=get_s3_objects(s3=Cloud_Object_Storage_Connection_client,bucket="smart-city-live-vehicle-positions", last_modified_min=starttime, last_modified_max=endtime)

## Reading the Vehicle Positions from the objects and store them into the DataFrame

In [7]:
def extract_data(df_temp):
    df_temp = df_temp['VP']
    df_temp = df_temp.dropna()
    df_temp = df_temp.tolist()
    df_temp = pd.DataFrame.from_records(df_temp)
    return df_temp

df_lvd = pd.DataFrame()
for obj in objs:
    df_lvd = df_lvd.append(extract_data(pd.read_json(Cloud_Object_Storage_Connection_client.get_object(Bucket='smart-city-live-vehicle-positions', Key=obj['Key'])['Body'].read(), lines=True)))   

df_lvd = df_lvd.reset_index(drop = True)

In [8]:
df_lvd

,acc,drst,loc,spd,line,jrn,dl,start,hdg,tsi,...,stop,occu,veh,desi,oper,odo,lat,oday,seq,label
0,0.0,0.0,GPS,0.00,588,8.0,299.0,10:04,237.0,1646899191,...,1453132,0,820,801,47,0.0,60.209848,2022-03-10,NaN,NaN
1,0.0,1.0,GPS,0.00,120,895.0,119.0,10:01,224.0,1646899191,...,1431183,0,1051,86,22,94.0,60.194654,2022-03-10,NaN,NaN
2,0.1,0.0,GPS,12.14,964,923.0,-129.0,09:14,63.0,1646899191,...,None,0,41,506,30,12966.0,60.233694,2022-03-10,NaN,NaN
3,0.5,NaN,GPS,12.08,769,57.0,-166.0,09:40,335.0,1646899191,...,None,0,1124,322,22,NaN,60.210779,2022-03-10,NaN,NaN
4,-0.1,NaN,GPS,38.12,284,9651.0,-50.0,09:40,25.0,1646899191,...,None,0,6317,R,90,NaN,60.367697,2022-03-10,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111027,0.0,1.0,GPS,0.00,244,452.0,-346.0,09:12,234.0,1646899313,...,1130113.0,0,1015,212,22,19654.0,60.170823,2022-03-10,NaN,NaN
111028,NaN,1.0,ODO,NaN,636,8636.0,59.0,09:28,NaN,1646899313,...,4530501.0,0,1035,P,90,25909.0,NaN,2022-03-10,NaN,NaN
111029,0.0,1.0,GPS,0.00,973,921.0,299.0,09:35,94.0,1646899309,...,1370108.0,0,833,61T,6,10204.0,60.241650,2022-03-10,NaN,NaN
111030,0.0,1.0,GPS,0.00,973,921.0,299.0,09:35,94.0,1646899310,...,1370108.0,0,833,61T,6,10204.0,60.241650,2022-03-10,NaN,NaN


In [9]:
df_lvd.shape

(111032, 24)

In [10]:
df_lvd.head()

,acc,drst,loc,spd,line,jrn,dl,start,hdg,tsi,...,stop,occu,veh,desi,oper,odo,lat,oday,seq,label
0,0.0,0.0,GPS,0.00,588,8.0,299.0,10:04,237.0,1646899191,...,1453132,0,820,801,47,0.0,60.209848,2022-03-10,NaN,NaN
1,0.0,1.0,GPS,0.00,120,895.0,119.0,10:01,224.0,1646899191,...,1431183,0,1051,86,22,94.0,60.194654,2022-03-10,NaN,NaN
2,0.1,0.0,GPS,12.14,964,923.0,-129.0,09:14,63.0,1646899191,...,None,0,41,506,30,12966.0,60.233694,2022-03-10,NaN,NaN
3,0.5,NaN,GPS,12.08,769,57.0,-166.0,09:40,335.0,1646899191,...,None,0,1124,322,22,NaN,60.210779,2022-03-10,NaN,NaN
4,-0.1,NaN,GPS,38.12,284,9651.0,-50.0,09:40,25.0,1646899191,...,None,0,6317,R,90,NaN,60.367697,2022-03-10,NaN,NaN


## Clean the DataFrame, where 'lat' or 'long' have NA values

In [11]:
df_lvd = df_lvd[df_lvd['lat'].notna() & df_lvd['long'].notna()]

## Deciding which featuers/columns are relevant
(https://digitransit.fi/en/developers/apis/4-realtime-api/vehicle-positions/) 

In [12]:
for column in df_lvd.columns:
    percent = (df_lvd[column].isna().sum()/len(df_lvd))*100
    print(column + " has this many NA values: " + str(percent) + "%")
    

acc has this many NA values: 4.624852762584805%
drst has this many NA values: 16.740779970232932%
loc has this many NA values: 0.0%
spd has this many NA values: 0.06756941844645123%
line has this many NA values: 3.760146826520084%
jrn has this many NA values: 4.5572833441383525%
dl has this many NA values: 9.742779659778847%
start has this many NA values: 0.0%
hdg has this many NA values: 1.4545686970972542%
tsi has this many NA values: 0.0%
dir has this many NA values: 0.0%
long has this many NA values: 0.0%
route has this many NA values: 0.0%
tst has this many NA values: 0.0%
stop has this many NA values: 58.72056393071395%
occu has this many NA values: 0.0%
veh has this many NA values: 0.0%
desi has this many NA values: 0.0%
oper has this many NA values: 0.0%
odo has this many NA values: 19.650830464676716%
lat has this many NA values: 0.0%
oday has this many NA values: 0.0%
seq has this many NA values: 95.49567647031968%
label has this many NA values: 99.94704018554197%


In [13]:
df_lvd.columns

Index(['acc', 'drst', 'loc', 'spd', 'line', 'jrn', 'dl', 'start', 'hdg', 'tsi',
       'dir', 'long', 'route', 'tst', 'stop', 'occu', 'veh', 'desi', 'oper',
       'odo', 'lat', 'oday', 'seq', 'label'],
      dtype='object')

> Decision: Deleting "seq" and "label" because >95% are NA. Also deleting "odo", because doc says it's values are not reliable. "oday", "line" and "jrn" for internal purposes. The rest stays, additional cleaning can be done by the "hackers"

In [14]:
if 'seq' in df_lvd.columns:
    df_lvd_cleaned = df_lvd.drop(['seq', 'label', 'odo', 'oday', 'jrn', 'line'], axis=1)
else:
    df_lvd_cleaned = df_lvd.drop(['label', 'odo', 'oday', 'jrn', 'line'], axis=1)
        

df_lvd_cleaned = df_lvd_cleaned.reset_index(drop=True)
print(df_lvd_cleaned.head())

   acc  drst  loc    spd     dl  start    hdg         tsi dir       long  \
0  0.0   0.0  GPS   0.00  299.0  10:04  237.0  1646899191   1  25.077927   
1  0.0   1.0  GPS   0.00  119.0  10:01  224.0  1646899191   1  25.031280   
2  0.1   0.0  GPS  12.14 -129.0  09:14   63.0  1646899191   2  25.030158   
3  0.5   NaN  GPS  12.08 -166.0  09:40  335.0  1646899191   1  24.888061   
4 -0.1   NaN  GPS  38.12  -50.0  09:40   25.0  1646899191   1  25.090595   

   route                       tst     stop  occu   veh desi  oper        lat  
0   1801  2022-03-10T07:59:51.305Z  1453132     0   820  801    47  60.209848  
1   1086  2022-03-10T07:59:51.309Z  1431183     0  1051   86    22  60.194654  
2   1506  2022-03-10T07:59:51.308Z     None     0    41  506    30  60.233694  
3   4322  2022-03-10T07:59:51.312Z     None     0  1124  322    22  60.210779  
4  3001R  2022-03-10T07:59:51.258Z     None     0  6317    R    90  60.367697  


## Important infos about the features:
- spd => in m/s
- acc => m/s^2
- dl => delay in sec
- drst => door status (0 = closed, 1 = open)
   

In [15]:
(df_lvd_cleaned['loc'] == "GPS").sum() / len(df_lvd_cleaned) * 100

95.1240446688642

In [16]:
df_lvd_cleaned = df_lvd_cleaned[df_lvd_cleaned['loc'] == "GPS"]
df_lvd_cleaned = df_lvd_cleaned.reset_index(drop=True)

In [17]:
df_lvd_cleaned = df_lvd_cleaned.drop(['loc'], axis=1)

In [18]:
df_lvd_cleaned.head()

,acc,drst,spd,dl,start,hdg,tsi,dir,long,route,tst,stop,occu,veh,desi,oper,lat
0,0.0,0.0,0.00,299.0,10:04,237.0,1646899191,1,25.077927,1801,2022-03-10T07:59:51.305Z,1453132,0,820,801,47,60.209848
1,0.0,1.0,0.00,119.0,10:01,224.0,1646899191,1,25.031280,1086,2022-03-10T07:59:51.309Z,1431183,0,1051,86,22,60.194654
2,0.1,0.0,12.14,-129.0,09:14,63.0,1646899191,2,25.030158,1506,2022-03-10T07:59:51.308Z,None,0,41,506,30,60.233694
3,0.5,NaN,12.08,-166.0,09:40,335.0,1646899191,1,24.888061,4322,2022-03-10T07:59:51.312Z,None,0,1124,322,22,60.210779
4,-0.1,NaN,38.12,-50.0,09:40,25.0,1646899191,1,25.090595,3001R,2022-03-10T07:59:51.258Z,None,0,6317,R,90,60.367697


## Converting the tst into a other timestamp format to join with Weather Data

In [19]:
import datetime

date = []
hour = []

for x in df_lvd_cleaned.tst:
    x = x[:-11]
    date_obj = datetime.datetime.strptime(x, '%Y-%m-%dT%H')
    date.append(str(date_obj.date()) + " " + str(date_obj.time()))
    
date

['2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:00:00',
 '2022-03-10 07:

In [20]:
df_lvd_cleaned['timestamp'] = date

## Loading and Joining Weather Data with LVD

In [25]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='1591931f-746f-4121-9254-fe78acb75225', project_access_token='p-05143ae15508ed3063f85a7a57bffd7d59f3eeca')
pc = project.project_context

project.save_data(data=df_lvd_cleaned.to_csv(index=False),file_name=str(dateloading)+'_only_lvd_'+str(filename_starttime) + '-' + str(filename_endtime)+'.csv',overwrite=True)

{'file_name': '2022-03-10_only_lvd_8-8.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'datentransformationlvd-donotdelete-pr-aoejnvmtwnetce',
 'asset_id': '65183b64-2caa-4dd0-93cd-29d09a9b4a02'}